In [249]:
import tensorflow as tf

from tensorflow.python.distribute import distribution_strategy_context as ds_context
from tensorflow.python.eager import context
from tensorflow.python.framework import ops
from tensorflow.python.framework import tensor_shape
from tensorflow.python.keras import activations
from tensorflow.python.keras import backend as K
from tensorflow.python.keras import constraints
from tensorflow.python.keras import initializers
from tensorflow.python.keras import regularizers
from tensorflow.python.keras.engine.base_layer import Layer
from tensorflow.python.keras.engine.input_spec import InputSpec
from tensorflow.python.keras.utils import generic_utils
from tensorflow.python.keras.utils import tf_utils
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import control_flow_ops
from tensorflow.python.ops import control_flow_util
from tensorflow.python.ops import math_ops
from tensorflow.python.ops import state_ops
from tensorflow.python.platform import tf_logging as logging
from tensorflow.python.training.tracking import base as trackable
from tensorflow.python.training.tracking import data_structures
from tensorflow.python.util import nest
from tensorflow.python.util.tf_export import keras_export
from tensorflow.tools.docs import doc_controls

class SLSTMCell(tf.keras.layers.Layer):
    """
    继承Layer并重载了5个函数，参照TF中LSTMCell源码并做了修改
    """
    
    def __init__(self,
                   units,
                   activation='tanh',
                   recurrent_activation='hard_sigmoid',
                   use_bias=True,
                   kernel_initializer='glorot_uniform',
                   recurrent_initializer='orthogonal',
                   bias_initializer='zeros',
                   unit_forget_bias=True,
                   kernel_regularizer=None,
                   recurrent_regularizer=None,
                   bias_regularizer=None,
                   kernel_constraint=None,
                   recurrent_constraint=None,
                   bias_constraint=None,
                   dropout=0.,
                   recurrent_dropout=0.,
                   implementation=1,
                   **kwargs):
        #print("init start")
        self._enable_caching_device = kwargs.pop('enable_caching_device', False)
        super(SLSTMCell, self).__init__(**kwargs)
        self.num_gates = 6
        self.units = units
        self.activation = activations.get(activation)
        self.recurrent_activation = activations.get(recurrent_activation)
        self.use_bias = use_bias

        self.kernel_initializer = initializers.get(kernel_initializer)
        self.recurrent_initializer = initializers.get(recurrent_initializer)
        self.bias_initializer = initializers.get(bias_initializer)
        self.unit_forget_bias = unit_forget_bias

        self.kernel_regularizer = regularizers.get(kernel_regularizer)
        self.recurrent_regularizer = regularizers.get(recurrent_regularizer)
        self.bias_regularizer = regularizers.get(bias_regularizer)

        self.kernel_constraint = constraints.get(kernel_constraint)
        self.recurrent_constraint = constraints.get(recurrent_constraint)
        self.bias_constraint = constraints.get(bias_constraint)

        self.dropout = min(1., max(0., dropout))
        self.recurrent_dropout = min(1., max(0., recurrent_dropout))
        if self.recurrent_dropout != 0 and implementation != 1:
            logging.debug(RECURRENT_DROPOUT_WARNING_MSG)
            self.implementation = 1
        else:
            self.implementation = implementation
        # tuple(_ListWrapper) was silently dropping list content in at least 2.7.10,
        # and fixed after 2.7.16. Converting the state_size to wrapper around
        # NoDependency(), so that the base_layer.__setattr__ will not convert it to
        # ListWrapper. Down the stream, self.states will be a list since it is
        # generated from nest.map_structure with list, and tuple(list) will work
        # properly.
        self.state_size = data_structures.NoDependency([self.units, self.units])
        self.output_size = self.units
        
        self.SeasonalGate = False
        
        #print("init end")

    
    
    @tf_utils.shape_type_conversion

    def build(self, input_shape):
        print("build start")
        input_dim = input_shape[-1]
        self.kernel = self.add_weight(
            # W_x: 用于和当前时刻隐层输入x做乘法的矩阵
            # 加入季节门，将self.units * 4改为self.units * 6 
            # 这里
            shape=(input_dim, self.units * self.num_gates),
            name='kernel',
            initializer=self.kernel_initializer,
            regularizer=self.kernel_regularizer,
            constraint=self.kernel_constraint)

        self.recurrent_kernel = self.add_weight(
            # W_h: 用于和前一时刻隐层输出h做乘法的矩阵
            # 加入季节门，将self.units * 4改为self.units * 6
            shape=(self.units, self.units * self.num_gates),
            name='recurrent_kernel',
            initializer=self.recurrent_initializer,
            regularizer=self.recurrent_regularizer,
            constraint=self.recurrent_constraint)


        if self.use_bias:
            if self.unit_forget_bias:

                def bias_initializer(_, *args, **kwargs):
                    return K.concatenate([
                          self.bias_initializer((self.units,), *args, **kwargs),
                          initializers.Ones()((self.units,), *args, **kwargs),
                          self.bias_initializer((self.units * (self.num_gates - 2),), *args, **kwargs),
                      ])
            else:
                bias_initializer = self.bias_initializer
            
            self.bias = self.add_weight(
                  shape=(self.units * self.num_gates,),
                  name='bias',
                  initializer=bias_initializer,
                  regularizer=self.bias_regularizer,
                  constraint=self.bias_constraint)

        else:
            self.bias = None
        self.built = True
        print("build end")


    def _compute_carry_and_output(self, x, h_tm1, c_tm1):
        """Computes carry and output using split kernels."""
        '''
        方案一（注意call中还有）
        通过短期信息对输入值进行分流，包括Wx+b向量的极差、标准化X、标准化X的极差
        
        模式1
        采用分离法计算各个门控单元的值
        本模式尝试将选通器的判断方法调整为h_tm1和x
        本方法中x,h_tm1,c_tm1均为各个门加mask后拼成的元组，故需要多传多
        模式1中的x是 Wx+b 
        '''
        x_i, x_f, x_c, x_o, x_s1, x_s2 = x
        h_tm1_i, h_tm1_f, h_tm1_c, h_tm1_o, h_tm1_s1, h_tm1_s2 = h_tm1
        i = self.recurrent_activation(
            x_i + K.dot(h_tm1_i, self.recurrent_kernel[:, :self.units]))
        f = self.recurrent_activation(x_f + K.dot(
            h_tm1_f, self.recurrent_kernel[:, self.units:self.units * 2]))
        
        # 季节门
        # 批差分
        sdB2 = 1.82
        sd = 2.41
        sdT2 = 3.58

        # 季节门1【改】
        s1 = K.tanh(x_s1 + K.dot(
            h_tm1_s1, self.recurrent_kernel[:, self.units * 4:self.units * 5]))
        # 季节门2【改】
        s2 = self.recurrent_activation(x_s2 + K.dot(
            h_tm1_s2, self.recurrent_kernel[:, self.units * 5:]))
        # 方案1*
        #SGate = K.max(x_s1) - K.min(x_s1) > 
        #if SGate:
        
        # 方案1--还有call中的一部分
        if self.SeasonalGate:
            
            c = f * c_tm1 * s1 + i * self.activation(x_c + K.dot(
                h_tm1_c, self.recurrent_kernel[:, self.units * 2:self.units * 3]))
            o = self.recurrent_activation(
                x_o + K.dot(h_tm1_o, self.recurrent_kernel[:, self.units * 3:self.units * 4]))
        else:
            c = f * c_tm1 * s2 + i * self.activation(x_c + K.dot(
                h_tm1_c, self.recurrent_kernel[:, self.units * 2:self.units * 3]))
            o = self.recurrent_activation(
                x_o + K.dot(h_tm1_o, self.recurrent_kernel[:, self.units * 3:self.units * 4]))

        return c, o

    def _compute_carry_and_output_fused(self, z, c_tm1):
        """
        方案二
        通过长期信息即细胞状态来分流
        
        模式2
        Computes carry and output using fused kernels.
        使用合并向量的计算方法
        """
        
        z0, z1, z2, z3, z4, z5 = z
        i = self.recurrent_activation(z0)
        f = self.recurrent_activation(z1)
        
        #季节门
        sdB4 = 0.2828
        sdB2 = 0.5102
        sd = 0.7975
        sdT2 = 0.9501
        #if True:
        if K.max(K.softmax(c_tm1)) - K.min(K.softmax(c_tm1)) > 0.7975: #【改】
            #季节门1【改】
            s = self.recurrent_activation(z4)
            #s = K.square(s)
        else:
            #季节门2【改】
            s = self.recurrent_activation(z5)
            
        #c = f * c_tm1 + i * self.activation(z2)
        c = f * c_tm1 * s + i * self.activation(z2)
        o = self.recurrent_activation(z3)
        return c, o

    def call(self, inputs, states, training=None):
        num_gates = 6
        h_tm1 = states[0]  # previous memory state
        c_tm1 = states[1]  # previous carry state

        #dp_mask = self.get_dropout_mask_for_cell(inputs, training, count=num_gates)
        
        #rec_dp_mask = self.get_recurrent_dropout_mask_for_cell(
        #    h_tm1, training, count=num_gates)

        if self.implementation == 1:
            '''
            implementation是计算模式，1或者2
            
            方案一（注意computer中还有）
            即inputs值大于k倍标准差，根据实际计算
            '''
            if  K.mean(inputs) > 1 : 
                self.SeasonalGate = True

            if 0 < self.dropout < 1.:
                inputs_i = inputs * dp_mask[0]
                inputs_f = inputs * dp_mask[1]
                inputs_c = inputs * dp_mask[2]
                inputs_o = inputs * dp_mask[3]
                inputs_s1 = inputs * dp_mask[4]
                inputs_s2 = inputs * dp_mask[5]
            else:
                inputs_i = inputs
                inputs_f = inputs
                inputs_c = inputs
                inputs_o = inputs
                inputs_s1 = inputs
                inputs_s2 = inputs
            k_i, k_f, k_c, k_o, k_s1, k_s2 = array_ops.split(
                  self.kernel, num_or_size_splits=6, axis=1)
            x_i = K.dot(inputs_i, k_i)
            x_f = K.dot(inputs_f, k_f)
            x_c = K.dot(inputs_c, k_c)
            x_o = K.dot(inputs_o, k_o)
            x_s1 = K.dot(inputs_o, k_s1)
            x_s2 = K.dot(inputs_o, k_s2)
            if self.use_bias:
                b_i, b_f, b_c, b_o, b_s1, b_s2 = array_ops.split(
                    self.bias, num_or_size_splits=6, axis=0)
                x_i = K.bias_add(x_i, b_i)
                x_f = K.bias_add(x_f, b_f)
                x_c = K.bias_add(x_c, b_c)
                x_o = K.bias_add(x_o, b_o)
                x_s1 = K.bias_add(x_o, b_s1)
                x_s2 = K.bias_add(x_o, b_s2)

            if 0 < self.recurrent_dropout < 1.:
                h_tm1_i = h_tm1 * rec_dp_mask[0]
                h_tm1_f = h_tm1 * rec_dp_mask[1]
                h_tm1_c = h_tm1 * rec_dp_mask[2]
                h_tm1_o = h_tm1 * rec_dp_mask[3]
                h_tm1_s1 = h_tm1 * rec_dp_mask[4]
                h_tm1_s2 = h_tm1 * rec_dp_mask[5]
            else:
                h_tm1_i = h_tm1
                h_tm1_f = h_tm1
                h_tm1_c = h_tm1
                h_tm1_o = h_tm1
                h_tm1_s1 = h_tm1
                h_tm1_s2 = h_tm1
            x = (x_i, x_f, x_c, x_o, x_s1, x_s2)
            h_tm1 = (h_tm1_i, h_tm1_f, h_tm1_c, h_tm1_o,h_tm1_s1,h_tm1_s2)
            c, o = self._compute_carry_and_output(x, h_tm1, c_tm1)
        else:
            # 模式2
            # 以下为使用随机遮挡的方法
            if 0. < self.dropout < 1.:
                inputs = inputs * dp_mask[0]
            
            z = K.dot(inputs, self.kernel)
            #不是z = K.dot(self.kernel, inputs)，因为x_t是行向量
            z += K.dot(h_tm1,self.recurrent_kernel)
            #不是z += K.dot(self.recurrent_kernel, h_tm1)，因为x_t是行向量
            if self.use_bias:
                z = K.bias_add(z, self.bias)
            
            z = array_ops.split(z, num_or_size_splits=num_gates, axis=1)
            c, o = self._compute_carry_and_output_fused(z, c_tm1)

        h = o * self.activation(c)
        return h, [h, c]
    
    def get_config(self):
        print("config start")

        config = {
            'units':
                self.units,
            'activation':
                activations.serialize(self.activation),
            'recurrent_activation':
                activations.serialize(self.recurrent_activation),
            'use_bias':
                self.use_bias,
            'kernel_initializer':
                initializers.serialize(self.kernel_initializer),
            'recurrent_initializer':
                initializers.serialize(self.recurrent_initializer),
            'bias_initializer':
                initializers.serialize(self.bias_initializer),
            'unit_forget_bias':
                self.unit_forget_bias,
            'kernel_regularizer':
                regularizers.serialize(self.kernel_regularizer),
            'recurrent_regularizer':
                regularizers.serialize(self.recurrent_regularizer),
            'bias_regularizer':
                regularizers.serialize(self.bias_regularizer),
            'kernel_constraint':
                constraints.serialize(self.kernel_constraint),
            'recurrent_constraint':
                constraints.serialize(self.recurrent_constraint),
            'bias_constraint':
                constraints.serialize(self.bias_constraint),
            'dropout':
                self.dropout,
            'recurrent_dropout':
                self.recurrent_dropout,
            'implementation':
                self.implementation
        }
        base_config = super(SLSTMCell, self).get_config()
        print("config end")
        return dict(list(base_config.items()) + list(config.items()))
        

    '''
    def get_initial_state(self, inputs=None, batch_size=None, dtype=None):
        return list(_generate_zero_filled_state_for_cell(
            self, inputs, batch_size, dtype))
    
    '''
    

# 航空数据集

In [256]:
import tensorflow as tf

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd

filename = "AirPassengers.csv"
f=open(filename,encoding='UTF-8')
#names=['Date','Num']
df = pd.read_csv(f,header=0)
df.columns = ['Date','AirPassengers']
#df = df.set_index(, drop = True)
#df = df.set_index("Date")
#print(data[0])
df.head()

,Date,AirPassengers
0,1,112
1,2,118
2,3,132
3,4,129
4,5,121


In [280]:

def univariate_data(dataset, 
                    start_index, end_index, 
                    history_size, target_size, 
                    single_step=True):
    '''
    dataset: 自变量
    start_index: 训练集或测试集的第一条所在行
    end_index: 训练集或测试集的最后一条所在行
    history_size: 输入网络的时间跨度，即RNN循环层长度
    target_size: 远期预测的时间跨度，若为下一期，则为0
    single_step: 是否单步预测
    '''
    data = []
    labels = []

    start_index = start_index + history_size
    if end_index is None:
        end_index = len(dataset) - target_size

    for i in range(start_index, end_index):
        indices = range(i-history_size, i)
        # Reshape data from (history_size,) to (history_size, 1)
        data.append(np.reshape(dataset[indices], (history_size, 1)))
        #labels.append(dataset[i+target_size])
        
        if single_step:
            labels.append(dataset[i+target_size])
        else:
            labels.append(dataset[i:i+target_size]) 
        
    return np.array(data), np.array(labels)

TRAIN_SPLIT = 144-12*3 #这里应该改为1220*0.8
tf.random.set_seed(13)
uni_data = df['AirPassengers']
uni_data.index = df['Date']
uni_data.head()
uni_data = uni_data.values
#print(uni_data[1])
#print(len(uni_data))
uni_train_mean = uni_data[:TRAIN_SPLIT].mean()
uni_train_std = uni_data[:TRAIN_SPLIT].std()
uni_data = (uni_data-uni_train_mean)/uni_train_std
#print(uni_data[0:24])
#print(uni_data[19])
#print(uni_data[20])

univariate_past_history = 12
univariate_future_target = 12
#若要单变量单步预测预测下一天，则为0,同时sigle_step=True
#单变量远期预测，若要跳过本周，则为5，同时sigle_step=True
#若要单变量多步预测预测下一天，则为0,同时sigle_step=False


def create_time_steps(length):
    return list(range(-length, 0))

BATCH_SIZE = 6
BUFFER_SIZE = 100



## 单步

In [271]:

x_train_uni, y_train_uni = univariate_data(uni_data, 0, TRAIN_SPLIT,
                                           univariate_past_history,
                                           univariate_future_target)
x_val_uni, y_val_uni = univariate_data(uni_data, TRAIN_SPLIT, None,
                                       univariate_past_history,
                                       univariate_future_target)

train_univariate = tf.data.Dataset.from_tensor_slices((x_train_uni, y_train_uni))
train_univariate = train_univariate.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()

val_univariate = tf.data.Dataset.from_tensor_slices((x_val_uni, y_val_uni))
val_univariate = val_univariate.batch(BATCH_SIZE).repeat()



In [272]:
print("验证集x:")
print(x_val_uni[0])
print("验证集y:")
print(y_val_uni[0])

验证集x:
[[1.28090293]
 [1.02261341]
 [1.53919244]
 [1.37482639]
 [1.55093288]
 [2.39624402]
 [3.05370824]
 [3.21807429]
 [2.03229061]
 [1.50397115]
 [0.92868995]
 [1.24568163]
 [1.51571158]
 [1.30438379]
 [2.05577147]
 [1.93836715]
 [2.22013753]
 [2.83064002]
 [3.72291289]
 [3.85205765]
 [2.72497613]
 [2.06751191]
 [1.53919244]
 [2.04403104]]
验证集y:
2.184916231728903


In [273]:
SLSTMLayer = tf.keras.layers.RNN(
    SLSTMCell(8,implementation=2,dropout=0.,use_bias=1), 
    input_shape = x_train_uni.shape[-2:], 
    dynamic=True)

In [274]:
slstm_model = tf.keras.models.Sequential([
    #tf.keras.layers.RNN(SLSTMCell(8,implementation=1,dropout=0.)),
    SLSTMLayer,
    #print("dense start")
    tf.keras.layers.Dense(1)
    #print("dense end")

])

build start
build end


In [275]:
slstm_model.compile(optimizer='adam', loss='mape')

In [277]:
for x, y in val_univariate.take(1):
    print(slstm_model.predict(x).shape)

EVALUATION_INTERVAL = 200 #这个值对训练结果的影响很大
EPOCHS = 10

# 注意：如不重新声明model，继续训练，则相当于在原有基础上继续训练；
# 如要重新训练，则须重新声明
slstm_history = slstm_model.fit(train_univariate, epochs=EPOCHS,
                      steps_per_epoch=EVALUATION_INTERVAL,
                      validation_data=val_univariate, validation_steps=12)

(6, 1)
Train for 200 steps, validate for 12 steps
Epoch 1/10
200/200 [==============================] - 31s 154ms/step - loss: 69.7979 - val_loss: 50.4201
Epoch 2/10
200/200 [==============================] - 27s 137ms/step - loss: 69.2265 - val_loss: 49.3211
Epoch 3/10
200/200 [==============================] - 27s 136ms/step - loss: 69.2753 - val_loss: 49.0367
Epoch 4/10
200/200 [==============================] - 26s 131ms/step - loss: 69.1686 - val_loss: 49.1167
Epoch 5/10
200/200 [==============================] - 28s 139ms/step - loss: 68.4605 - val_loss: 48.8711
Epoch 6/10
200/200 [==============================] - 34s 170ms/step - loss: 68.1363 - val_loss: 48.6718
Epoch 7/10
200/200 [==============================] - 28s 139ms/step - loss: 67.7699 - val_loss: 47.4589
Epoch 8/10
200/200 [==============================] - 39s 195ms/step - loss: 67.6383 - val_loss: 49.3110
Epoch 9/10
200/200 [==============================] - 51s 254ms/step - loss: 67.8021 - val_loss: 48.6637
Epoch

In [278]:
validation_predict = np.array([])
validation_real = np.array([])

for x, y in val_univariate.take(int((36-univariate_past_history)/BATCH_SIZE)):
    res = slstm_model.predict(x)
    res = np.transpose(res)#转置
    y = np.transpose(y)
    print(y.shape)
    validation_predict = np.concatenate([validation_predict, res[0]])#向量连接
    validation_real = np.concatenate([validation_real, y])
    
print(validation_predict.shape) 
#之所以只有2批，因为测试集中刨去look_back=60, 1220-800-60=360，而批容量为256；
#当批容量为25，则为360/
#print(res)
print(validation_real.shape)
def recover_data(data, mean, std):
    res = data * std + mean
    return res
validation_predict = recover_data(validation_predict, uni_train_mean, uni_train_std)
validation_real = recover_data(validation_real, uni_train_mean, uni_train_std)
print(validation_predict[1])
print(validation_real[1])
import pandas as pd
#data = np.array([np.transpose(validation_predict), np.transpose(validation_real)])
data = np.array([validation_predict, validation_real])
data = np.transpose(data)
print(data.shape)
#slstm_validation_result = pd.DataFrame(data, columns=c("predict","real"))
slstm_validation_result = pd.DataFrame(data)
print(slstm_validation_result.head())



(6,)
(6,)
(12,)
(12,)
316.3952110789336
391.0
(12, 2)
            0      1
0  311.926378  417.0
1  316.395211  391.0
2  306.574195  419.0
3  315.489188  461.0
4  338.409919  472.0


In [279]:
slstm_validation_result.to_csv("AIR24_slstm_M2L24_2sd_tanh_single.csv",index=False)


## 多步

In [281]:
past_history = univariate_past_history
future_target = univariate_future_target
dataset = uni_data
x_train_multi, y_train_multi = univariate_data(dataset, 0,
                                                TRAIN_SPLIT, past_history,
                                                 future_target, single_step=True)
x_val_multi, y_val_multi = univariate_data(dataset,
                                             TRAIN_SPLIT, None, past_history,
                                             future_target,single_step=True)

In [284]:
print ('Single window of past history : {}'.format(x_val_multi.shape))
print ('\nTarget temperature to predict : {}'.format(y_val_multi.shape))

Single window of past history : (12, 12, 1)

Target temperature to predict : (12,)


In [285]:
train_data_multi = tf.data.Dataset.from_tensor_slices((x_train_multi, y_train_multi))
train_data_multi = train_data_multi.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()

val_data_multi = tf.data.Dataset.from_tensor_slices((x_val_multi, y_val_multi))
val_data_multi = val_data_multi.batch(BATCH_SIZE).repeat()

In [307]:
y_val_multi.shape

(12,)

In [287]:
def multi_step_plot(history, true_future, prediction):
    plt.figure(figsize=(12, 6))
    num_in = create_time_steps(len(history))
    num_out = len(true_future)

    plt.plot(num_in, np.array(history), label='History')
    plt.plot(np.arange(num_out), np.array(true_future), 'bo',
           label='True Future')
    if prediction.any():
        plt.plot(np.arange(num_out), np.array(prediction), 'ro',
             label='Predicted Future')
    plt.legend(loc='upper left')
    plt.show()

In [289]:
# 多步预测使用两层LSTM，第一次为32维输出，第二层为16层输出
# 模式2为c, 模式1为x或h
multi_step_model = tf.keras.models.Sequential([
    tf.keras.layers.RNN(
        SLSTMCell(32,implementation=1,dropout=0.,use_bias=1),
        return_sequences=True,
        input_shape = x_train_multi.shape[-2:], 
        dynamic=True),
    tf.keras.layers.RNN(
        SLSTMCell(16,implementation=1,dropout=0.,use_bias=1), #可对activation进行调整
        dynamic=True),
    tf.keras.layers.Dense(future_target)

])

build start
build end
build start
build end


In [298]:
# 多步预测使用两层LSTM，第一次为32维输出，第二层为16层输出
# 模式2为c, 模式1为x或h
multi_step_model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(32,
        return_sequences=True,
        input_shape = x_train_multi.shape[-2:]),
    tf.keras.layers.LSTM(16),
    tf.keras.layers.Dense(future_target)

])

In [299]:
'''
multi_step_model = tf.keras.models.Sequential()
multi_step_model.add(tf.keras.layers.LSTM(32,
                                          return_sequences=True,
                                          input_shape=x_train_multi.shape[-2:]))
multi_step_model.add(tf.keras.layers.LSTM(16, activation='relu'))
multi_step_model.add(tf.keras.layers.Dense(5))
'''
#multi_step_model.compile(optimizer=tf.keras.optimizers.RMSprop(clipvalue=1.0), 
#                    loss='mae')
multi_step_model.compile(optimizer='adam', loss='mape')


In [300]:
for x, y in val_data_multi.take(2):
    print (multi_step_model.predict(x).shape)
#查看预测结果的形状

(6, 12)
(6, 12)


In [301]:
EPOCHS=10
EVALUATION_INTERVAL = 300

In [302]:
multi_step_history = multi_step_model.fit(train_data_multi, epochs=EPOCHS,
                                          steps_per_epoch=EVALUATION_INTERVAL,
                                          validation_data=val_data_multi,
                                          validation_steps=60)

Train for 300 steps, validate for 60 steps
Epoch 1/10
300/300 [==============================] - 15s 49ms/step - loss: 96.2112 - val_loss: 54.8109
Epoch 2/10
300/300 [==============================] - 7s 23ms/step - loss: 71.7590 - val_loss: 56.9798
Epoch 3/10
300/300 [==============================] - 7s 24ms/step - loss: 67.8146 - val_loss: 44.8000
Epoch 4/10
300/300 [==============================] - 7s 23ms/step - loss: 66.2911 - val_loss: 54.2609
Epoch 5/10
300/300 [==============================] - 8s 28ms/step - loss: 63.7994 - val_loss: 54.7302
Epoch 6/10
300/300 [==============================] - 8s 27ms/step - loss: 61.6711 - val_loss: 50.6481
Epoch 7/10
300/300 [==============================] - 8s 27ms/step - loss: 60.3887 - val_loss: 58.2656
Epoch 8/10
300/300 [==============================] - 7s 24ms/step - loss: 58.4904 - val_loss: 52.4835
Epoch 9/10
300/300 [==============================] - 8s 27ms/step - loss: 58.0779 - val_loss: 51.2139
Epoch 10/10
300/300 [========

In [40]:
plot_train_history(multi_step_history, 'AirPassengers SLSTM Multi-Steps Training and validation loss')

NameError: name 'plot_train_history' is not defined

In [305]:
# 数据导出
# 多步预测的全部结果
'''
#之所以只取2个，因为只有2批
#shape：因为测试集中刨去look_back=60, 1220-800-60-5=355 ，
#为什么不是355+1=356？因为没有最后一条，也就是没有最后五天的预测，
#但由于是多步预测，所以本质上只少了最后一天的预测
#但为什么256-99会得到360？批次之间的序号问题导致的！应先合并再提取！
#print(res)
# 420-look_back-target = 类接力的维数
'''
validation_predictAll = np.array([])
validation_realAll = np.array([])
numVali = 1

times = int((144-TRAIN_SPLIT-univariate_past_history-future_target)/BATCH_SIZE)
for x, y in val_data_multi.take(times):
    
    res = multi_step_model(x)
    #res = np.transpose(res)#转置
    print(res.shape)
    #y = np.transpose(y)
    print(y.shape)
    if numVali == 1:
        validation_predictAll = res
        validation_realAll = y
    else:     
        validation_predictAll = np.vstack((validation_predictAll, res))
        validation_realAll = np.vstack((validation_realAll, y))
    numVali = numVali + 1

print(validation_predictAll.shape)
print(validation_realAll.shape)

# 多步预测的部分结果-转化为类接力集合
validation_predict = np.array([])
validation_real = np.array([])
for i in range(validation_predictAll.shape[0]):
    if i % future_target == 0:
        validation_predict = np.concatenate([validation_predict, validation_predictAll[i]])#行向量连接
        validation_real = np.concatenate([validation_real, validation_realAll[i]])

print(validation_predict.shape) 
print(validation_real.shape)
#应为testSize-lookBack-target=420-60-5=355, 缺最后1组target的预测值

def recover_data(data, mean, std):
    res = data * std + mean
    return res

validation_predict = recover_data(validation_predict, uni_train_mean, uni_train_std)
validation_real = recover_data(validation_real, uni_train_mean, uni_train_std)
print(validation_predict[1])
print(validation_real[1])

import pandas as pd
#data = np.array([np.transpose(validation_predict), np.transpose(validation_real)])
data = np.array([validation_predict, validation_real])
data = np.transpose(data)
print(data.shape)
lstm_validation_result = pd.DataFrame(data)
print(lstm_validation_result.head())


(6, 12)
(6,)
(6, 12)
(6,)
(12, 12)
(2, 6)
(12,)
(6,)
316.5201122245654
391.0
(2,)
                                                   0
0  [315.3316627455753, 316.5201122245654, 321.380...
1         [417.0, 391.0, 419.0, 461.0, 472.0, 535.0]


In [33]:
lstm_validation_result.to_csv("AIR_lstm_multi.csv",index=False)